In [ ]:
import json
import utils
import pandas as pd

In [ ]:
def format_for_candranker(gazname, unique_placenames_array):
    """
    This function returns the unique alternate names in a given gazetteer
    in the format required by DeezyMatch candidate ranker."""
    with open(gazname + ".txt", "w") as fw:
        for pl in unique_placenames_array:
            pl = pl.strip()
            if pl:
                fw.write(pl.strip() + "\t0\tfalse\n")

In [ ]:
counter = 0

queries = []
incrid = []
title = []
toponyms = []
contextwords = []
redirected = []
bhocontent = []
with open('bho_parsed.json') as f:
    
    my_dict = json.load(f)
    new_dict = dict()
    for k in my_dict:
        toponym_altnames = []
        context_words = []
        content = []
        is_redirected = True
        place_name = my_dict[k]["place_name"]
        description = my_dict[k]["description"]
        # Process only redirections:
        if not "—See " in place_name:
            toponym_altnames, context_words = utils.preprocess_title(place_name)
            is_redirected = False
            content = utils.process_content(description)
        
        incrid.append(k)
        title.append(place_name)
        toponyms.append(toponym_altnames)
        contextwords.append(context_words)
        redirected.append(is_redirected)
        bhocontent.append(content)
        
        for t in toponym_altnames:
            queries.append(t)

queries = list(set(queries))
format_for_candranker("../toponym_matching/gazetteers/bho_queries", queries)

bhodf = pd.DataFrame()
bhodf["id"] = incrid
bhodf["title"] = title
bhodf["toponyms"] = toponyms
bhodf["contextwords"] = contextwords
bhodf["redirected"] = redirected
bhodf["content"] = bhocontent
bhodf.to_csv("bho.csv", index=False)